In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import mne
import scipy.io
import shutil
from scipy.signal import resample, resample_poly, decimate

In [2]:
# Hyper Params
oldFS = 256
newFS = 32
length = 8 * newFS # n*newfs [eikahne n hosse amder window length]
stride = length
p = 0.5  # eita use kortesi jate akta nirdisto row sum korar por jodi value  p ke extend kore tahole annotation 1 kore dibo

In [3]:
s=pd.read_csv('eldfs/eeg1.csv')

In [4]:
f=s.iloc[:,0:18]

In [8]:
f

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-9.856820e-05,-1.010096e-04,-1.507532e-04,-9.933114e-05,-8.285166e-05,-7.461192e-05,1.999250e-05,-1.431238e-04,-8.742929e-05,-7.735850e-05,-1.391565e-04,-9.762117e-06,-8.422495e-05,-6.164196e-05,-7.320713e-06,-1.278651e-04,-5.813244e-05,-1.408350e-04
1,-9.887338e-05,-1.011622e-04,-1.524317e-04,-1.005518e-04,-8.163096e-05,-7.674815e-05,2.029767e-05,-1.466333e-04,-8.926035e-05,-7.827403e-05,-1.414454e-04,-1.159317e-05,-6.179455e-05,-6.286266e-05,-1.006729e-05,-1.290858e-04,-6.148937e-05,-1.432764e-04
2,-9.719491e-05,-9.963631e-05,-1.516687e-04,-9.948373e-05,-8.178355e-05,-7.751109e-05,1.983991e-05,-1.452601e-04,-8.803964e-05,-7.766368e-05,-1.583826e-04,-1.235611e-05,-7.812144e-05,-6.179455e-05,-1.128800e-05,-1.275599e-04,-6.164196e-05,-1.420557e-04
3,-9.872079e-05,-1.011622e-04,-1.536524e-04,-1.020777e-04,-8.346201e-05,-7.995249e-05,1.983991e-05,-1.472437e-04,-8.987070e-05,-7.888438e-05,-1.377833e-04,-1.037247e-05,-6.942394e-05,-6.347301e-05,-1.144058e-05,-1.303065e-04,-6.301525e-05,-1.440394e-04
4,-9.780526e-05,-1.003993e-04,-1.516687e-04,-1.010096e-04,-8.010508e-05,-7.949473e-05,1.953473e-05,-1.457178e-04,-8.880258e-05,-7.796885e-05,-1.428187e-04,-1.083023e-05,-7.323863e-05,-6.423595e-05,-1.220352e-05,-1.289332e-04,-6.362560e-05,-1.429712e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962811,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09
962812,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09
962813,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09
962814,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09


In [9]:
data_annotation=s['annotation']
data=pd.concat([f,data_annotation],axis=1)

In [10]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,annotation
0,-9.856820e-05,-1.010096e-04,-1.507532e-04,-9.933114e-05,-8.285166e-05,-7.461192e-05,1.999250e-05,-1.431238e-04,-8.742929e-05,-7.735850e-05,-1.391565e-04,-9.762117e-06,-8.422495e-05,-6.164196e-05,-7.320713e-06,-1.278651e-04,-5.813244e-05,-1.408350e-04,0
1,-9.887338e-05,-1.011622e-04,-1.524317e-04,-1.005518e-04,-8.163096e-05,-7.674815e-05,2.029767e-05,-1.466333e-04,-8.926035e-05,-7.827403e-05,-1.414454e-04,-1.159317e-05,-6.179455e-05,-6.286266e-05,-1.006729e-05,-1.290858e-04,-6.148937e-05,-1.432764e-04,0
2,-9.719491e-05,-9.963631e-05,-1.516687e-04,-9.948373e-05,-8.178355e-05,-7.751109e-05,1.983991e-05,-1.452601e-04,-8.803964e-05,-7.766368e-05,-1.583826e-04,-1.235611e-05,-7.812144e-05,-6.179455e-05,-1.128800e-05,-1.275599e-04,-6.164196e-05,-1.420557e-04,0
3,-9.872079e-05,-1.011622e-04,-1.536524e-04,-1.020777e-04,-8.346201e-05,-7.995249e-05,1.983991e-05,-1.472437e-04,-8.987070e-05,-7.888438e-05,-1.377833e-04,-1.037247e-05,-6.942394e-05,-6.347301e-05,-1.144058e-05,-1.303065e-04,-6.301525e-05,-1.440394e-04,0
4,-9.780526e-05,-1.003993e-04,-1.516687e-04,-1.010096e-04,-8.010508e-05,-7.949473e-05,1.953473e-05,-1.457178e-04,-8.880258e-05,-7.796885e-05,-1.428187e-04,-1.083023e-05,-7.323863e-05,-6.423595e-05,-1.220352e-05,-1.289332e-04,-6.362560e-05,-1.429712e-04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962811,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,0
962812,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,0
962813,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,0
962814,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,3.500000e-09,0


In [11]:
len(data)

962816

In [12]:
def downsample(data, oldFS, newFS):
    data_annotation=data['annotation']
    data_clip=data.iloc[:,0:18]
    data=pd.concat([data_clip,data_annotation],axis=1)
    newNumSamples = int((data.shape[0] / oldFS) * newFS)
    newData = pd.DataFrame(resample(data[data.columns[:-1]], newNumSamples))
    annotation_indices = list(range(0, len(data), 8))
    annotation = data.annotation.loc[annotation_indices].reset_index(drop = True)
    newData['annotation'] = annotation
    return newData

In [15]:
%%time

path = 'eldfs/'
babydfs = list()
for file in sorted(os.listdir(path)):
    print(file)
    df = downsample(pd.read_csv(path + file), oldFS, newFS)
    finaldfs = list()
    for i in range(0, len(df), stride):
        annotation = 0
        try:
            if df[df.columns[-1]].iloc[i:i+length].value_counts()[1] > int(p * length):
                annotation = 1
        except:
            annotation = 0
        int_dfs = list()
        for j in range(18):
            window = df[df.columns[j]].iloc[i:i+length]
            int_dfs.append(window)
        int_df = pd.DataFrame(pd.concat(int_dfs, axis = 0, ignore_index = True)).T
        int_df['annotation'] = annotation

        finaldfs.append(int_df)
    finaldf = pd.concat(finaldfs, axis = 0)
    babydfs.append(finaldf)
babydf = pd.concat(babydfs, axis = 0).reset_index(drop = True).dropna(how = 'any')

eeg0.csv
eeg1.csv
eeg11.csv
eeg13.csv
eeg15.csv
eeg16.csv
eeg17.csv
eeg25.csv
eeg29.csv
eeg3.csv
eeg31.csv
eeg32.csv
eeg33.csv
eeg34.csv
eeg35.csv
eeg36.csv
eeg37.csv
eeg38.csv
eeg39.csv
eeg4.csv


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [16]:
babydf

,0,1,2,3,4,5,6,7,8,9,...,4599,4600,4601,4602,4603,4604,4605,4606,4607,annotation
0,-3.263788e-07,-1.235657e-06,-2.262840e-06,-7.633439e-08,-2.164782e-07,-9.267774e-07,-2.608645e-07,4.420569e-07,1.074870e-06,1.201442e-06,...,-1.410432e-07,1.537603e-08,-8.363894e-08,-3.257639e-07,-2.275977e-07,-8.146320e-08,-1.897746e-07,-1.462715e-07,1.954133e-08,0
1,-7.209614e-09,3.681703e-06,3.559992e-06,5.172217e-06,3.584803e-06,1.639444e-06,7.919758e-07,7.644812e-06,1.409753e-05,1.424207e-05,...,-2.108430e-08,-6.075327e-08,3.585114e-08,-7.522059e-08,-2.129476e-07,1.840603e-08,-4.448791e-08,-1.414767e-07,-1.127482e-07,0
2,-2.486152e-05,-2.503058e-05,-2.186363e-05,-2.480244e-05,-2.744258e-05,-3.197565e-05,-2.888203e-05,-2.740444e-05,-2.505538e-05,-2.380489e-05,...,2.480865e-08,2.674820e-09,-1.229418e-07,-1.181962e-07,4.533225e-09,-8.686495e-08,1.657593e-09,2.014616e-07,-2.392995e-08,0
3,6.060522e-06,7.697736e-06,9.215949e-06,1.084153e-05,1.033151e-05,1.047530e-05,7.125132e-06,5.205420e-06,5.480814e-06,5.544259e-06,...,-3.678916e-08,4.569411e-08,-1.483502e-09,-1.193529e-07,8.563138e-08,-8.082530e-08,6.143709e-08,7.245130e-08,-1.086324e-08,0
4,-2.500956e-07,3.655730e-07,1.434406e-06,1.338836e-06,1.395847e-06,1.193354e-06,4.941691e-07,-1.154113e-06,-1.678423e-06,-2.962719e-06,...,-1.177482e-08,2.158331e-07,1.414320e-07,2.125777e-07,1.735678e-07,1.709235e-08,-4.350430e-08,1.600691e-07,3.397057e-08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38823,-8.874981e-06,-8.567710e-06,-7.482840e-06,-7.108656e-06,-5.753267e-06,-4.379086e-06,-5.035435e-06,-5.029869e-06,-4.787151e-06,-4.696357e-06,...,1.167609e-06,4.388172e-07,8.032826e-07,5.066306e-07,4.639861e-07,4.091424e-07,-1.768839e-06,-1.995077e-06,-3.525502e-06,0
38824,-1.470589e-06,-2.534388e-06,-1.701899e-06,1.054022e-06,1.112566e-06,2.784270e-06,2.682054e-06,1.907659e-06,1.636055e-06,1.755764e-06,...,-5.321980e-06,-6.370271e-06,-5.702493e-06,-6.331531e-06,-6.396456e-06,-7.795904e-06,-8.898536e-06,-7.187432e-06,-7.602793e-06,0
38825,1.959206e-05,1.983068e-05,2.022580e-05,2.052560e-05,2.312558e-05,2.396794e-05,2.443081e-05,2.133596e-05,2.264722e-05,2.308864e-05,...,-1.251002e-07,1.325776e-07,-1.239752e-07,1.313204e-07,-1.226322e-07,1.299237e-07,-1.212041e-07,1.284802e-07,-1.197563e-07,0
38826,9.130242e-08,-8.282140e-08,9.036598e-08,-8.193430e-08,8.952457e-08,-8.113509e-08,8.876424e-08,-8.041054e-08,8.807261e-08,-7.974916e-08,...,7.141023e-06,8.936959e-06,8.597389e-06,8.308195e-06,8.024005e-06,8.644325e-06,1.003760e-05,9.195006e-06,9.691872e-06,0


In [11]:
babydf.to_csv('babydf8sec_19_row.csv')

KeyboardInterrupt: 

In [ ]:
babydf_copy=babydf

In [ ]:
babydf = babydf_copy.dropna(how = 'any')
len(babydf)

In [ ]:
 babydf